In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_APIKEY")
llm=ChatGroq(api_key=api_key,model_name="meta-llama/llama-4-scout-17b-16e-instruct")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002715D461AD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002715D5ADBD0>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

Stuff Documents

In [3]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
loader=PyMuPDFLoader("apjspeech.pdf")
docs=loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [4]:
template=''' 
Write a concise and a short summary of the following speech: {text}
'''
prompt=PromptTemplate(input_variables=['text'],template=template)


In [5]:
from langchain.chains.summarize import load_summarize_chain


In [6]:
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary=chain.run(docs)
output_summary  

C:\Users\DeLL\AppData\Local\Temp\ipykernel_8456\3974195537.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summary=chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Write a concise and a short summary of the following speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I 
enjoyed every minute of my tenure enriched by the wonderful association from each one 
of you, hailing from different walks of life, be it politics, science and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and student 
community who are the future wealth of our country. During my interaction at 
Rashtrapati Bhavan in Delhi and at every state and union territory as well as through my 
online interactions, 

"**Summary of A P J Abdul Kalam's Departing Speech**\n\nFormer President of India, A P J Abdul Kalam, delivered a farewell speech after completing his five-year term. He expressed his gratitude to the people of India and shared his experiences and lessons learned during his tenure. Kalam highlighted 10 key messages, including:\n\n1. **Accelerating development**: Fulfilling the aspirations of the youth, who desire to live in a developed India by 2020.\n2. **Empowering villages**: Transforming 600,000 villages into empowered and connected entities.\n3. **Mobilizing rural core competence**: Encouraging rural areas to develop their own competitive strengths.\n4. **Seed to Food**: Focusing on agricultural growth and empowering farmers.\n5. **Defeating problems**: Overcoming challenges through positive attitude and partnership.\n\nKalam also shared inspiring stories of individuals and communities who have made a positive impact, including:\n\n* **Vidwan Coimbatore SR Krishna Murthy**: A pers

MAP REDUCE


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [8]:
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [9]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [10]:
len(final_documents)

13

In [11]:
chunk_prompt='''
Please summarize the below speech:
Speech:'{text}'
Summary:
'''
map_prompt=PromptTemplate(input_variable=['text'],template=chunk_prompt)

In [12]:
final_prompt='''
Provide a final summary of the entire speech with these important points.
Add a motivation title and start the precise summary with an introduction.
Speech:{text}
'''
final_prompt_temp=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_temp

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide a final summary of the entire speech with these important points.\nAdd a motivation title and start the precise summary with an introduction.\nSpeech:{text}\n')

In [13]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=final_prompt_temp,
    verbose=True
)
output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:'A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I 
enjoyed every minute of my tenure enriched by the wonderful association from each one 
of you, hailing from different walks of life, be it politics, science and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and student 
community who are the future wealth of our country. During my interaction at 
Rashtrapati Bhavan in Delhi and at every state and union territory as well as through my 
online interactions, I have many uni


> Finished chain.


c:\Users\DeLL\OneDrive\Documents\coding\python\LangChain\TextSummarizer\summarizer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (2042 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:

Provide a final summary of the entire speech with these important points.
Add a motivation title and start the precise summary with an introduction.
Speech:Here is a summary of the speech:

Former President of India, A.P.J. Abdul Kalam, is addressing the nation on his departure after completing five years in office. He expresses his gratitude and delight in having worked with people from all walks of life. He shares his experiences and highlights 10 key messages that he learned during his tenure, which include:

1. Accelerating development to meet the aspirations of the youth
2. Empowering villages
3. Mobilizing rural resources for competitiveness
4. Focusing on agriculture from seed to food
5. Overcoming problems through determination and partnership
6. Having courage in the face of calamities
7. Improving connectivity for societal transformation
8. Defending the nation with pride
9. Encouraging a youth movement for a develo

"**Motivation to Empower a Developed India: A Vision for a Prosperous and Secure Nation**\n\n**Introduction**\n\nThe former President of India, A.P.J. Abdul Kalam, delivered a speech reflecting on his five-year tenure in office. He expressed his gratitude to the people of India and shared his experiences, highlighting 10 key messages that he learned during his presidency. The speech emphasized the importance of empowering villages, accelerating development, and overcoming challenges to create a prosperous and secure India.\n\n**Summary of the Speech**\n\nDr. Kalam's speech focused on several key areas, including:\n\n1. **Empowering Villages**: He emphasized the need to empower villages and provide urban amenities in rural areas to create a prosperous and developed India. He cited the example of Periyar PURA, a successful rural development project that generated employment, created entrepreneurs, and improved healthcare and education in 65 villages.\n\n2. **Accelerating Development**: D